Selector的用法

前面介绍过利用BeautifulSoup和PyQuery以及正则表达式来提取网页数据，非常方便。而Scrapy也有自己的提取数据的方法，即Selector选择器。Select是基于lxml来构建的，支持XPath选择器、CSS选择器以及正则表达式，功能齐全，解析速度和准确度非常高。

本来可以采用scrapy shell来调试选择器的使用方法。也可以直接使用Selector模块直接模拟。官网也提供了相应的方法：http://scrapy-chs.readthedocs.io/zh_CN/latest/topics/selectors.html 。

关于XPath的语法和运算符可以参考:http://www.runoob.com/xpath/xpath-tutorial.html

# 1. 两种选择器

由于在response中使用XPath、CSS查询十分普遍，因此，Scrapy提供了两个实用的快捷方式: response.xpath() 及 response.css():

In [2]:
from scrapy import Selector
html='''
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>
'''
selector = Selector(text=html)
print(selector.xpath('//title/text()'))
print(selector.css('title::text'))

[<Selector xpath='//title/text()' data='Example website'>]
[<Selector xpath='descendant-or-self::title/text()' data='Example website'>]


如你所见， .xpath() 及 .css() 方法返回一个类 SelectorList 的实例, 它是一个新选择器的列表。这个API可以用来快速的提取嵌套数据。

## 1.1 提取数据

为了提取真实的原文数据，你需要调用 .extract()和extract_first() 方法如下:

In [3]:
print(selector.xpath('//title/text()').extract())
print(selector.css('title::text').extract_first())

['Example website']
Example website


extract()返回的是一个列表，extract_first()返回单个值。

## 1.2 根据属性来提取

In [8]:
from scrapy import Selector
html='''
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='123image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>
'''
selector = Selector(text=html)

现在我们想获取base标签上的链接：

In [4]:
print(selector.xpath('//base/@href').extract_first())
print(selector.css('base::attr("href")').extract_first())

http://example.com/
http://example.com/


获取图片链接：

In [6]:
print(selector.xpath('//div[@id="images"]/a/@href').extract())
print(selector.css('#images a::attr("href")').extract())

['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']
['image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']


In [9]:
#print(selector.xpath('//a[contains(@href, "image")]/@href').extract())
print(selector.css('a[href*=image]::attr("href")').extract())

['123image1.html', 'image2.html', 'image3.html', 'image4.html', 'image5.html']


获取img标签内的src内容：

In [10]:
print(selector.xpath('//a[contains(@href, "image")]/img/@src').extract())
print(selector.css('a[href*=image] img::attr(src)').extract())

['image1_thumb.jpg', 'image2_thumb.jpg', 'image3_thumb.jpg', 'image4_thumb.jpg', 'image5_thumb.jpg']
['image1_thumb.jpg', 'image2_thumb.jpg', 'image3_thumb.jpg', 'image4_thumb.jpg', 'image5_thumb.jpg']


# 2.嵌套选择器

选择器方法( .xpath() 或 .css() )返回相同类型的选择器列表，因此你也可以对这些选择器调用选择器方法。

In [ ]:
from scrapy import Selector
html='''
<html>
 <head>
  <base href='http://example.com/' />
  <title>Example website</title>
 </head>
 <body>
  <div id='images'>
   <a href='image1.html'>Name: My image 1 <br /><img src='image1_thumb.jpg' /></a>
   <a href='image2.html'>Name: My image 2 <br /><img src='image2_thumb.jpg' /></a>
   <a href='image3.html'>Name: My image 3 <br /><img src='image3_thumb.jpg' /></a>
   <a href='image4.html'>Name: My image 4 <br /><img src='image4_thumb.jpg' /></a>
   <a href='image5.html'>Name: My image 5 <br /><img src='image5_thumb.jpg' /></a>
  </div>
 </body>
</html>
'''
selector = Selector(text=html)

In [12]:
links = selector.xpath('//a[contains(@href, "image")]')
print(links)
for index, link in enumerate(links):
    args = (index, link.css('::attr(href)').extract_first(), link.xpath('img/@src').extract_first())
    print ('链接%d指向url是：%s  和图是：%s' % args)

[<Selector xpath='//a[contains(@href, "image")]' data='<a href="123image1.html">Name: My image '>, <Selector xpath='//a[contains(@href, "image")]' data='<a href="image2.html">Name: My image 2 <'>, <Selector xpath='//a[contains(@href, "image")]' data='<a href="image3.html">Name: My image 3 <'>, <Selector xpath='//a[contains(@href, "image")]' data='<a href="image4.html">Name: My image 4 <'>, <Selector xpath='//a[contains(@href, "image")]' data='<a href="image5.html">Name: My image 5 <'>]
链接0指向url是：123image1.html  和图是：image1_thumb.jpg
链接1指向url是：image2.html  和图是：image2_thumb.jpg
链接2指向url是：image3.html  和图是：image3_thumb.jpg
链接3指向url是：image4.html  和图是：image4_thumb.jpg
链接4指向url是：image5.html  和图是：image5_thumb.jpg


# 3. 结合正则表达式

Selector 也有一个 .re() 方法，用来通过正则表达式来提取数据。然而，不同于使用 .xpath() 或者 .css() 方法, .re() 方法返回unicode字符串的列表。所以你无法构造嵌套式的 .re() 调用。

In [15]:
selector.xpath('//a[contains(@href, "image")]/text()').extract()
selector.xpath('//a[contains(@href, "image")]/text()').re_first(r'Name:\s(.*)')

'My image 1 '